# Time Series Forecasting

## Read in Libraries

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.base.datetools import dates_from_str
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults

## Forecast Process

In [61]:
d = "3Q19" # Set this equal to the file path you're working out of (current quarter)
m = {"OC", "SV", "SD", "CHI", "WLA"} # These are the markets we are forecasting

for i in m:
    
    # Read in data - including a quick operation to clean up the Excel
    xl = pd.read_excel(f'.../{d}/0_Inputs/{d}DataViewOFC.xlsx',sheet_name=i, usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
    xl.drop(xl.tail(24).index,inplace=True) # drop forecast
    
    #Here we create our calculated variables we will use in forecasting
    xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #stationary
    xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #stationary
    xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
    xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #stationary
    xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #non-stationary
    xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
    xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) #non-stationary 
    xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #stationary
    xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #stationary
    xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #stationary
    
    xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast
    
    # Here we want to create and export a df of normalized data that can drive ICOP LEI output
    normalized_xl = xl
    normalized_xl.drop(normalized_xl.head(48).index,inplace=True) # Drop older less reliable data
    normalized_xl=(normalized_xl-normalized_xl.mean())/normalized_xl.std()
    normalized_xl = normalized_xl.drop(normalized_xl.columns[[0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13]], axis=1) # Drop data we don't need
    
    # We have to invert certain normalized values
    normalized_xl['Vacancy Rate (%)'] = normalized_xl['Vacancy Rate (%)'].apply(lambda x: x*-1)
    normalized_xl['Comps MA %'] = normalized_xl['Comps MA %'].apply(lambda x: x*-1)
    
    # Then we can export to Excel for further analysis
    from openpyxl import load_workbook

    book = load_workbook(f".../{d}/1_Outputs/Historical_All.xlsx")
    writer = pd.ExcelWriter(f".../{d}/1_Outputs/Historical_All.xlsx", engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    normalized_xl.to_excel(writer, i)

    writer.save()
    
    # Back to the Forecasting Process
    xl.to_csv(f".../{d}/1_Outputs/{i}/{i}Historical.csv") #Save down full series with added variables
    
    # Here we prep the data that will go into the first model, select the model, fit the model, and export the model
    data = xl[['Revenue Growth', 'OFC Emp Growth', 'Abs MA %', 'Vacancy Rate (%)', 'Comps MA %']]
    model = VAR(data)
    results = model.fit(maxlags=4, ic='bic')
    
    fv = results.fittedvalues
    fv.to_csv(f".../{d}/1_Outputs/{i}/{i}BICFitted.csv")
    
    lag_order = results.k_ar
    a = np.asarray(results.forecast(data.values[-lag_order:], 40))
    np.savetxt(f".../{d}/1_Outputs/{i}/{i}OFCforecastBIC.csv", a, delimiter=",")
    
    # Repeat for a different optimization
    results = model.fit(maxlags=4, ic='aic')
    
    fv = results.fittedvalues
    fv.to_csv(f".../{d}/1_Outputs/{i}/{i}AICFitted.csv")
    lag_order = results.k_ar
    a = np.asarray(results.forecast(data.values[-lag_order:], 40))
    np.savetxt(f".../{d}/1_Outputs/{i}/{i}OFCforecastAIC.csv", a, delimiter=",")
    
    # Run next class of models
    series = xl[['Revenue Growth']]
    
    res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')
    res.aic_min_order
    res.bic_min_order
    p = np.asarray(res.aic_min_order[0])
    q = np.asarray(res.aic_min_order[1])
    
    model = ARIMA(series, order = (p,0,q))
    results_AR = model.fit(disp=0)
    
    fv = pd.DataFrame(results_AR.fittedvalues)
    fv.to_csv(f".../{d}/1_Outputs/{i}/{i}ARMAAICFitted.csv")
    
    df_forecast = pd.DataFrame(results_AR.forecast(40))
    df_forecast.T.to_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastARIMA1.csv")
    
    # Repeat for a different optimization
    p = np.asarray(res.bic_min_order[0])
    q = np.asarray(res.bic_min_order[1])
    
    model = ARIMA(series, order = (p,0,q))
    results_AR = model.fit(disp=0)
    
    fv = pd.DataFrame(results_AR.fittedvalues)
    fv.to_csv(f".../{d}/1_Outputs/{i}/{i}ARMABICFitted.csv")
    df_forecast = pd.DataFrame(results_AR.forecast(40))
    df_forecast.T.to_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastARIMA2.csv")
    
    # Now we need to combine all forecasts into one file
    files = {
        "date": pd.read_csv(f".../{d}/0_Inputs/Date.csv", squeeze=True),
        "VARAIC": pd.read_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastAIC.csv", usecols=[0], header=None, names=["VARAIC"], squeeze=True),
        "ARIMA1": pd.read_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
        "ARIMA2": pd.read_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
        "VARBIC": pd.read_csv(f".../{d}/1_Outputs/{i}/{i}OFCforecastBIC.csv", usecols=[0], header=None, names=["VARBIC"], squeeze=True)
    }
    
    df_i = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)
    df_i = df_i.set_index("Date")
    df_i['AVERAGE'] = df_i.mean(axis=1)
    
    # Final forecast Excel workbook
    from openpyxl import load_workbook

    book = load_workbook(f".../{d}/1_Outputs/RevPAF Forecasts.xlsx")
    writer = pd.ExcelWriter(f".../{d}/1_Outputs/RevPAF Forecasts.xlsx", engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    df_i.to_excel(writer, i)

    writer.save()


C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continu

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users